In [1]:
%pip install beautifulsoup4 requests pandas -q

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import requests
from bs4 import BeautifulSoup as bs

In [3]:
res = requests.get('https://myanimelist.net/anime/season')
soup = bs(res.text, 'html.parser')

In [4]:
titles = []
genres = []
releases = []
total_episodes = []
duration = []
members = []
score = []
div_animes_season = soup.find('div', class_='seasonal-anime-list js-seasonal-anime-list js-seasonal-anime-list-key-1')
for div in div_animes_season.find_all('div', class_='js-anime-category-producer seasonal-anime js-seasonal-anime js-anime-type-all js-anime-type-1'):
    titles.append(div.find('span', class_='js-title').text)
    genres.append(str([g.text.replace('\n', '') for g in div.find('div', class_='js-genre-inner')]))
    info = div.find('div', class_='info').text.replace(',', '').split()
    releases.append(f'{info[0]} {info[1]} {info[2]}')
    total_episodes.append(info[3] + ' ' + info[4])
    duration.append(info[5] + ' ' + info[6])
    scores_info = div.find('div', class_='scormem-container').text.replace('\n', '').split()
    members.append(scores_info[1])
    score.append(float(scores_info[0]) if scores_info[0] != 'N/A' else 0)

In [5]:
import pandas as pd
df = pd.DataFrame({'Title': titles, 'Genre': genres, 'Release': releases, 'Total Episodes': total_episodes, 'Duration': duration, 'Members': members, 'Score': score})
df.head()

,Title,Genre,Release,Total Episodes,Duration,Members,Score
0,Chainsaw Man,"['Action', 'Supernatural']",Oct 12 2022,12 eps,24 min,803K,8.88
1,Spy x Family Part 2,"['Action', 'Comedy']",Oct 1 2022,13 eps,24 min,477K,8.68
2,Mob Psycho 100 III,"['Action', 'Comedy', 'Supernatural']",Oct 6 2022,? eps,23 min,374K,8.66
3,Boku no Hero Academia 6th Season,['Action'],Oct 1 2022,25 eps,23 min,337K,8.21
4,Bleach: Sennen Kessen-hen,"['Action', 'Adventure', 'Fantasy']",Oct 11 2022,? eps,24 min,279K,9.08


In [6]:
df.Genre = df.Genre.str.strip('[]')
df.Genre = df.Genre.str.replace("'", '')
df.head()

,Title,Genre,Release,Total Episodes,Duration,Members,Score
0,Chainsaw Man,"Action, Supernatural",Oct 12 2022,12 eps,24 min,803K,8.88
1,Spy x Family Part 2,"Action, Comedy",Oct 1 2022,13 eps,24 min,477K,8.68
2,Mob Psycho 100 III,"Action, Comedy, Supernatural",Oct 6 2022,? eps,23 min,374K,8.66
3,Boku no Hero Academia 6th Season,Action,Oct 1 2022,25 eps,23 min,337K,8.21
4,Bleach: Sennen Kessen-hen,"Action, Adventure, Fantasy",Oct 11 2022,? eps,24 min,279K,9.08


In [7]:
df.to_csv('SeasonAnimes.csv', index=False)